In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
import gmaps
# Google developer API key
from config import gkey
from datetime import date
# Configure gmaps
gmaps.configure(api_key=gkey)
from pprint import pprint
import itertools



In [2]:
#data provided by emily and harlan https://github.com/NABSA/gbfs/blob/master/systems.csv
bigdata_df=pd.read_csv("big_data.csv")
bigdata_df.head()

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
0,AE,ADCB Bikeshare,"Abu Dhabi, AE",ABU,https://www.bikeshare.ae/,https://api-core.bikeshare.ae/gbfs/gbfs.json
1,AE,Careem BIKE,"Dubai, AE",careem_bike,https://www.careem.com/en-ae/careem-bike/,https://dubai.publicbikesystem.net/ube/gbfs/v1/
2,AR,Ecobici,"Buenos Aires, AR",bike_buenosaires,https://www.buenosaires.gob.ar/ecobici,https://buenosaires.publicbikesystem.net/ube/g...
3,AR,MiBiciTuBici,"Rosario, Santa Fe, AR",biketobike,https://www.mibicitubici.gob.ar/,https://www.mibicitubici.gob.ar/opendata/gbfs....
4,AR,Movo Buenos Aires,"Buenos Aires, AR",movo_bue,https://movo.me/ar,https://gbfs.movo.me/v1/buenosaires/gbfs.json


In [3]:
#getting data from only US cities
us_data=bigdata_df.loc[(bigdata_df["Country Code"]=="US")]
us_data

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
331,US,Austin B-cycle,"Austin, TX",bcycle_austin,http://austinbcycle.com,https://gbfs.bcycle.com/bcycle_austin/gbfs.json
332,US,Aventura BCycle,"Aventura, FL",bcycle_aventura,https://aventura.bcycle.com,https://gbfs.bcycle.com/bcycle_aventura/gbfs.json
333,US,Bay Wheels,"San Francisco Bay Area, CA",BA,https://www.baywheels.com/,https://gbfs.baywheels.com/gbfs/gbfs.json
334,US,Bike Chattanooga,"Chattanooga, TN",bike_chattanooga,http://www.bikechattanooga.com/,https://chat.publicbikesystem.net/ube/gbfs/v1/
335,US,BikeLNK,"Lincoln, NE",bcycle_bikelnk,https://bikelnk.bcycle.com,https://gbfs.bcycle.com/bcycle_bikelnk/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [4]:
#getting data for bird and lime escooter
data_scoot = us_data.loc[us_data['System ID'].str.contains("bird", case=False)]
len(data_scoot)
data_scoot


,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
339,US,Bird Alexandria,"Alexandria, VA",bird-alexandria,https://www.bird.co,https://mds.bird.co/gbfs/alexandria/gbfs.json
340,US,Bird Chicago,"Chicago, IL",bird-chicago,https://www.bird.co,https://mds.bird.co/gbfs/chicago/gbfs.json
341,US,Bird Cleveland,"Cleveland, OH",bird-cleveland,https://www.bird.co,https://mds.bird.co/gbfs/cleveland/gbfs.json
342,US,Bird Columbus,"Columbus, OH",bird-columbus,https://www.bird.co,https://mds.bird.co/gbfs/columbus/gbfs.json
343,US,Bird Culver City,"Culver City, CA",bird-culver-city,https://www.bird.co,https://mds.bird.co/gbfs/culver-city/gbfs.json
344,US,Bird Detriot,"Detroit, MI",bird-detroit,https://www.bird.co,https://mds.bird.co/gbfs/detroit/gbfs.json
345,US,Bird Fairfax,"Fairfax, VA",bird-fairfax,https://www.bird.co,https://mds.bird.co/gbfs/fairfax/gbfs.json
346,US,Bird Indianapolis,"Indianapolis,IN",bird-indianapolis,https://www.bird.co,https://mds.bird.co/gbfs/indianapolis/gbfs.json
347,US,Bird Kansas City,"Kansas City, MO",bird-kansas-city,https://www.bird.co,https://mds.bird.co/gbfs/kansas-city/gbfs.json
348,US,Bird Los Angeles,"Los Angeles, CA",bird-los-angeles,https://www.bird.co,https://mds.bird.co/gbfs/los-angeles/gbfs.json


In [ ]:
#get bike coord for one bike
base_url= "https://mds.bird.co/gbfs/alexandria/gbfs.json"
result = requests.get(base_url).json()
base_url1=data["data"]["en"]["feeds"][2]["url"]
result1 = requests.get(base_url1).json()
bike_coord = result1['data']['bikes']
    bike_coord

In [21]:
#getting latitudes and longitudes for city as well as coordinates of bird scooters in each cities
all_url=data_scoot["Auto-Discovery URL"]
data_json = []
city_list = []
count_list = []
lat_list=[]
lon_list=[]
city_list_coord = []
x = 0

for i, row in data_scoot.iterrows():
    
    # set base_url
    base_url= row['Auto-Discovery URL']
    
    # set cities
    cities = row['Location']
    
    print(f'running record {x} on {cities}')
    
    result = requests.get(base_url).json()
    
    # get the URL from the city URL
    base_url1=result["data"]["en"]["feeds"][2]["url"]
    
    #target_url
    result1 = requests.get(base_url1).json()
    
    # get bike coordinates 
    bike_coord = result1['data']['bikes']
    
    print(f'there are {len(bike_coord)} bikes in record {x}')
    city_list.append(cities)
    count_list.append(len(bike_coord))
    
    # get lat and lon and city name into lists
    for bike in bike_coord:
        
        # Add Lat info to list
        lat_list.append([bike][0]['lat'])
        lon_list.append([bike][0]['lon'])
        city_list_coord.append(cities)
      
    x = x+1
    #set time sleep
    time.sleep(2)


https://mds.bird.co/gbfs/alexandria/gbfs.json
running record 0 on Alexandria, VA
there are 140 bikes in record 0
https://mds.bird.co/gbfs/chicago/gbfs.json
running record 1 on Chicago, IL
there are 45 bikes in record 1
https://mds.bird.co/gbfs/cleveland/gbfs.json
running record 2 on Cleveland, OH
there are 315 bikes in record 2
https://mds.bird.co/gbfs/columbus/gbfs.json
running record 3 on Columbus, OH
there are 229 bikes in record 3
https://mds.bird.co/gbfs/culver-city/gbfs.json
running record 4 on Culver City, CA
there are 26 bikes in record 4
https://mds.bird.co/gbfs/detroit/gbfs.json
running record 5 on Detroit, MI
there are 315 bikes in record 5
https://mds.bird.co/gbfs/fairfax/gbfs.json
running record 6 on Fairfax, VA
there are 56 bikes in record 6
https://mds.bird.co/gbfs/indianapolis/gbfs.json
running record 7 on Indianapolis,IN
there are 945 bikes in record 7
https://mds.bird.co/gbfs/kansas-city/gbfs.json
running record 8 on Kansas City, MO
there are 791 bikes in record 8
htt

In [27]:
#to save city level based on Geofence
df1 = pd.DataFrame({'City': city_list, 'Count':count_list})

City     Alexandria, VAChicago, ILCleveland, OHColumbus...
Count                                                 9362
dtype: object

In [25]:
#to retrieve geocode from Gmaps
lat_list = []
lon_list = [] 

for x in df1["City"]:
    # Target city
    target_city = x
    
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}"

    geo_data = requests.get(target_url).json()

    # Extract latitude and longitude
    lat_list.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    lon_list.append(geo_data["results"][0]["geometry"]["location"]["lng"])

In [28]:
#Save city level e-scooter counts to csv
df1["lat"] = lat_list
df1['lon']= lon_list
df1['Operator'] = "Lime"
today = date.today()
df1['Date'] = today
df1.to_csv('Spin_Lat_Lon.csv')
df1

,City,Count,lat,lon,Operator,Date
0,"Alexandria, VA",140,38.804836,-77.046921,Lime,2021-05-17
1,"Chicago, IL",45,41.878114,-87.629798,Lime,2021-05-17
2,"Cleveland, OH",315,41.499320,-81.694361,Lime,2021-05-17
3,"Columbus, OH",229,39.961176,-82.998794,Lime,2021-05-17
4,"Culver City, CA",26,34.021122,-118.396467,Lime,2021-05-17
5,"Detroit, MI",315,42.331427,-83.045754,Lime,2021-05-17
6,"Fairfax, VA",56,38.846224,-77.306373,Lime,2021-05-17
7,"Indianapolis,IN",945,39.768403,-86.158068,Lime,2021-05-17
8,"Kansas City, MO",791,39.099727,-94.578567,Lime,2021-05-17
9,"Los Angeles, CA",3263,34.052234,-118.243685,Lime,2021-05-17


In [102]:
# del df['geofencing_zone_id']
# del df['zone_area']
df.rename(columns={'lat':'Lat', 'lon':'Lon'})
df
# geocleaned_df = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list})
# df['Operator'] = "Bird"
today = date.today()
df['Date'] = today
df
df.to_csv('Bird_Lat_Lon.csv')

In [83]:
# zip location list
locations = list(zip(lat_list,lon_list))
locations

0     324
1     606
2      38
3      73
4      75
5      57
6      36
7     224
8     224
9      75
10     10
11     91
12      7
13    164
14     26
Name: Count, dtype: int64

In [90]:
locations = df[['lat','lon']]
locations
count = df["Count"]
count

0     324
1     606
2      38
3      73
4      75
5      57
6      36
7     224
8     224
9      75
10     10
11     91
12      7
13    164
14     26
Name: Count, dtype: int64

In [91]:
#generating the heat map using the city(lat and lon values) with
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,  weights = count,
                                 dissipating = False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))